In [ ]:
!pip install sentence_transformers
!pip install dgl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.4 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=794f806d1bee64fa0d45db67cf2427114c968c2ee4bff2227dc40980f6c233ea
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 16.4 MB/s eta 0:00:00


In [1]:
import json
# from sentence_transformers import SentenceTransformer
from lxml import html, etree
import torch
import torch.nn.functional as F
import dgl
from copy import deepcopy

In [ ]:
class Tree:
    def __init__(self, h_size):
        self.dgl_graph = dgl.DGLGraph()
        self.h_size = h_size

    def add_node(self, parent_id=None, tensor:torch.Tensor = torch.Tensor()):
        self.dgl_graph.add_nodes(1, data={'x': tensor.unsqueeze(0),
                                          'h': tensor.new_zeros(size=(1, self.h_size)),
                                          'c': tensor.new_zeros(size=(1, self.h_size))})
        added_node_id = self.dgl_graph.number_of_nodes() - 1
        if parent_id:
            self.dgl_graph.add_edges(added_node_id, parent_id)
        return added_node_id

    def add_node_bottom_up(self, child_ids, tensor: torch.Tensor):
        self.dgl_graph.add_nodes(1, data={'x': tensor.unsqueeze(0),
                                          'h': tensor.new_zeros(size=(1, self.h_size)),
                                          'c': tensor.new_zeros(size=(1, self.h_size))})
        added_node_id = self.dgl_graph.number_of_nodes() - 1
        for child_id in child_ids:
            self.dgl_graph.add_edges(child_id, added_node_id)
        return added_node_id

    def add_link(self, child_id, parent_id):
        self.dgl_graph.add_edges(child_id, parent_id)


class BatchedTree:
    def __init__(self, tree_list):
        graph_list = []
        for tree in tree_list:
            graph_list.append(tree.dgl_graph)
        self.batch_dgl_graph = dgl.batch(graph_list)

    def get_hidden_state(self):
        graph_list = dgl.unbatch(self.batch_dgl_graph)
        hidden_states = []
        max_nodes_num = max([graph.num_nodes() for graph in graph_list])
        for graph in graph_list:
            hiddens = graph.ndata['h']
            node_num, hidden_num = hiddens.size()
            if len(hiddens) < max_nodes_num:
                padding = hiddens.new_zeros(size=(max_nodes_num - node_num, hidden_num))
                hiddens = torch.cat((hiddens, padding), dim=0)
            hidden_states.append(hiddens)
        return torch.stack(hidden_states)


class TreeLSTM(torch.nn.Module):
    def __init__(self,
                 x_size,
                 h_size,
                 dropout,
                 cell_type='n_ary',
                 n_ary=None,
                 num_stacks=2):
        super(TreeLSTM, self).__init__()
        self.x_size = x_size
        self.dropout = torch.nn.Dropout(dropout)
        if cell_type == 'n_ary':
            self.cell = NaryTreeLSTMCell(n_ary, x_size, h_size)
        else:
            self.cell = ChildSumTreeLSTMCell(x_size, h_size)
        self.num_stacks = num_stacks
        self.linear = torch.nn.Linear(h_size, 7)

    def forward(self, batch: BatchedTree):
        batches = [deepcopy(batch) for _ in range(self.num_stacks)]
        for stack in range(self.num_stacks):
            cur_batch = batches[stack]
            if stack > 0:
                prev_batch = batches[stack - 1]
                cur_batch.batch_dgl_graph.ndata['x'] = prev_batch.batch_dgl_graph.ndata['h']
            # cur_batch.batch_dgl_graph.register_message_func(self.cell.message_func)
            # cur_batch.batch_dgl_graph.register_reduce_func(self.cell.reduce_func)
            # cur_batch.batch_dgl_graph.register_apply_node_func(self.cell.apply_node_func)
            cur_batch.batch_dgl_graph.ndata['iou'] = self.cell.W_iou(self.dropout(batch.batch_dgl_graph.ndata['x']))
            dgl.prop_nodes_topo(cur_batch.batch_dgl_graph, self.cell.message_func, self.cell.reduce_func, apply_node_func = self.cell.apply_node_func)
            h = cur_batch.get_hidden_state()
        logits = self.linear(h)

        return batches, logits




class NaryTreeLSTMCell(torch.nn.Module):
    def __init__(self, n_ary, x_size, h_size):
        super(NaryTreeLSTMCell, self).__init__()
        self.n_ary = n_ary
        self.h_size = h_size
        self.W_iou = torch.nn.Linear(x_size, 3 * h_size, bias=False)
        self.U_iou = torch.nn.Linear(n_ary * h_size, 3 * h_size, bias=False)
        self.b_iou = torch.nn.Parameter(torch.zeros(1, 3 * h_size), requires_grad=True)
        self.U_f = torch.nn.Linear(n_ary * h_size, n_ary * h_size)

    def message_func(self, edges):
        return {'h': edges.src['h'], 'c': edges.src['c']}

    def reduce_func(self, nodes):
        h_cat = nodes.mailbox['h'].view(nodes.mailbox['h'].size(0), -1)
        padding_hs = self.n_ary - nodes.mailbox['h'].size(1)
        padding = h_cat.new_zeros(size=(nodes.mailbox['h'].size(0), padding_hs * self.h_size))
        h_cat = torch.cat((h_cat, padding), dim=1)
        f = torch.sigmoid(self.U_f(h_cat)).view(nodes.mailbox['h'].size(0), self.n_ary, self.h_size)
        padding_cs = self.n_ary - nodes.mailbox['c'].size(1)
        padding = h_cat.new_zeros(size=(nodes.mailbox['c'].size(0), padding_cs, self.h_size))
        c = torch.cat((nodes.mailbox['c'], padding), dim=1)
        c = torch.sum(f * c, 1)
        return {'iou': nodes.data['iou'] + self.U_iou(h_cat), 'c': c}

    def apply_node_func(self, nodes):
        iou = nodes.data['iou'] + self.b_iou
        i, o, u = torch.chunk(iou, 3, 1)
        i, o, u = torch.sigmoid(i), torch.sigmoid(o), torch.tanh(u)
        c = i * u + nodes.data['c']
        h = o * torch.tanh(c)
        return {'h': h, 'c': c}


class ChildSumTreeLSTMCell(torch.nn.Module):
    def __init__(self, x_size, h_size):
        super(ChildSumTreeLSTMCell, self).__init__()
        self.W_iou = torch.nn.Linear(x_size, 3 * h_size, bias=False)
        self.U_iou = torch.nn.Linear(h_size, 3 * h_size, bias=False)
        self.b_iou = torch.nn.Parameter(torch.zeros(1, 3 * h_size), requires_grad=True)
        self.U_f = torch.nn.Linear(h_size, h_size)

    def message_func(self, edges):
        return {'h': edges.src['h'], 'c': edges.src['c']}

    def reduce_func(self, nodes):
        h_tild = torch.sum(nodes.mailbox['h'], 1)
        f = torch.sigmoid(self.U_f(nodes.mailbox['h']))
        c = torch.sum(f * nodes.mailbox['c'], 1)
        return {'iou': nodes.data['iou'] + self.U_iou(h_tild), 'c': c}

    def apply_node_func(self, nodes):
        iou = nodes.data['iou'] + self.b_iou
        i, o, u = torch.chunk(iou, 3, 1)
        i, o, u = torch.sigmoid(i), torch.sigmoid(o), torch.tanh(u)
        c = i * u + nodes.data['c']
        h = o * torch.tanh(c)
        return {'h': h, 'c': c}

In [ ]:
model = SentenceTransformer('sentence-transformers/all-roberta-large-v1', device='cuda')
H_SIZE = 30
X_SIZE = 1024

In [2]:
import torch
class Node:
    def __init__(self, parent_id = None):
        self.children = []
        self.data = ""
        self.embedding = None
        self.id = None
        self.parent_id = parent_id
        self.xpath = ""
    def add_child(self, child):
        self.children.append(child)
    def set_data(self, tag, text):
        self.data = ""
        if tag is etree.Comment:
            self.data += "comment"
        elif tag != None:
            self.data += tag

        if text != None:
            self.data += text

    def set_id(self, id):
        self.id = id



def loadData():
    f = open('data.json', 'r')
    val = f.read()
    f.close()
    d = json.loads(val)
    return d

def idTree(htmlTree):
    nodeIds = dict()
    count = 0
    for i in htmlTree.getiterator():
        nodeIds[i] = count
        count += 1
    return nodeIds

def convertCodeToTree(code):
    t = html.fromstring(code)
    r = etree.ElementTree(t)

    tree = Node()

    nodeIds = idTree(t)

    # Want dict mapping object -> id


    def parseTree(htmlTree, treeObject):
        tree_id = nodeIds[htmlTree]
        treeObject.set_data(htmlTree.tag, htmlTree.text)
        treeObject.set_id(tree_id)
        treeObject.xpath = r.getpath(htmlTree)

        for treeChild in htmlTree.getchildren():
            new_tree = Node(parent_id = tree_id)
            parseTree(treeChild, new_tree)
            treeObject.add_child(new_tree)
        return treeObject

    return parseTree(t, tree)


def embbedNode(nodes):
    nodes.embedding =  torch.tensor(model.encode([nodes.data])[0])
    for i in nodes.children:
        embbedNode(i)

def makeTree(t):
    finTree = Tree(H_SIZE)


    def addNodes(root):
        finTree.add_node(parent_id = root.parent_id, tensor=root.embedding)
        for i in root.children:
            addNodes(i)

    addNodes(t)
    return finTree

def batchTrees(treeList):
    return BatchedTree(treeList)

def trainLSTM(dataTrain, dataTest, labelTrain, labelTest):

    model = TreeLSTM(x_size=X_SIZE, h_size=H_SIZE, dropout=0.3, cell_type='child_sum', num_stacks=1)
    optimizer = torch.optim.Adagrad(model.parameters(),
                          lr=0.01,
                          weight_decay=0.0001)

    for epoch in range(50):

      b, logits = model(dataTrain)

      logp = F.log_softmax(logits, 2)
      logp = torch.reshape(logp, (logp.shape[0]*logp.shape[1], logp.shape[2]))
      labels = torch.reshape(labelTrain, (-1,))
      loss = F.nll_loss(logp, labels, reduction='sum')
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      pred = torch.argmax(logits, 2)
      pred = torch.reshape(pred, (-1,))

      accTrain = float(torch.sum(torch.eq(labels, pred))) / len(labels)

      with torch.no_grad():
        b, logits = model(dataTest)

        logp = F.log_softmax(logits, 2)
        logp = torch.reshape(logp, (logp.shape[0]*logp.shape[1], logp.shape[2]))
        labels = torch.reshape(labelTest, (-1,))
        pred = torch.argmax(logits, 2)
        pred = torch.reshape(pred, (-1,))

        accVal = float(torch.sum(torch.eq(labels, pred))) / len(labels)

      print("Epoch {:05d} | Loss {:.4f} | Acc Train {:.4f} | Acc Test {:.4f}".format(
          epoch, loss.item(), accTrain, accVal))
    return model(dataTrain)




In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

data = loadData()
# treeList = []
# for i in data['urls'][0:10]:
#     print(i, len(data['urls']))
#     trep = convertCodeToTree(data['code'][i])
#     embbedNode(trep)
#     print(trep)
#     lstmTree = makeTree(trep)
#     treeList.append(lstmTree)



In [ ]:
import pickle

# with open ('drive/MyDrive/treeList.pkl', 'rb') as f:
#   pickle.dump(treeList, f)

with open ('drive/MyDrive/treeList.pkl', 'rb') as f:
   treeList = pickle.load(f)

In [ ]:
labelsTrain = []
labelsTest = []
# l1 = list(data['urls'].values())[:2]

# Finding the maximum length of the labels
# maxL = max(len(labels) for labels in l1)

maxL = max([len(data['labels'][i]) for i in data['urls']])
maxL2 = 2393

print(maxL)
for i, url in enumerate(data['urls']):
  lab = data['labels'][url]
  print(i, len(lab))


  # l.append(len(lab))

  # lab = F.one_hot(torch.tensor(lab))
  if i < 100:
    lab2 = lab + [6 for i in range(maxL - len(lab))]
    labelsTrain.append(lab2)
  else:
    lab2 = lab + [6 for i in range(maxL2 - len(lab))]
    labelsTest.append(lab2)

# numClasses = 7
# for count, i in enumerate(labels):
#   node_num = l[count]
#   if (node_num < maxL):
#     padding = torch.tensor(())
#     padding = i.new_zeros(size=(maxL - node_num, numClasses))
#     labels[count] = torch.cat((i, padding), dim=0)




# labels = torch.stack(labels, dim=0)
labelsTrain = torch.tensor(labelsTrain)
labelsTest = torch.tensor(labelsTest)

In [ ]:
trainTreeList = treeList[0:100]
testTreeList = treeList[100::]
print(len(testTreeList))
print(labelsTest.shape)
print(max([len(data['labels'][i]) for i in data['urls'][100::]]))
for tree in testTreeList:
  print(tree.dgl_graph)
lstmTreeBatchTrain = batchTrees(trainTreeList)
lstmTreeBatchTest = batchTrees(testTreeList)

20
torch.Size([20, 2393])
2393
Graph(num_nodes=2217, num_edges=2214,
      ndata_schemes={'x': Scheme(shape=(1024,), dtype=torch.float32), 'h': Scheme(shape=(30,), dtype=torch.float32), 'c': Scheme(shape=(30,), dtype=torch.float32)}
      edata_schemes={})
Graph(num_nodes=1887, num_edges=1884,
      ndata_schemes={'x': Scheme(shape=(1024,), dtype=torch.float32), 'h': Scheme(shape=(30,), dtype=torch.float32), 'c': Scheme(shape=(30,), dtype=torch.float32)}
      edata_schemes={})
Graph(num_nodes=1936, num_edges=1933,
      ndata_schemes={'x': Scheme(shape=(1024,), dtype=torch.float32), 'h': Scheme(shape=(30,), dtype=torch.float32), 'c': Scheme(shape=(30,), dtype=torch.float32)}
      edata_schemes={})
Graph(num_nodes=2242, num_edges=2239,
      ndata_schemes={'x': Scheme(shape=(1024,), dtype=torch.float32), 'h': Scheme(shape=(30,), dtype=torch.float32), 'c': Scheme(shape=(30,), dtype=torch.float32)}
      edata_schemes={})
Graph(num_nodes=2235, num_edges=2232,
      ndata_schemes={'x': S

In [ ]:
model, l = trainLSTM(lstmTreeBatchTrain, lstmTreeBatchTest, labelsTrain, labelsTest)

/usr/local/lib/python3.10/dist-packages/dgl/core.py:82: DGLWarning: The input graph for the user-defined edge function does not contain valid edges
  dgl_warning(


Epoch 00000 | Loss 458588.2188 | Acc Train 0.4604 | Acc Test 0.4699
Epoch 00001 | Loss 435650.3125 | Acc Train 0.4631 | Acc Test 0.5703
Epoch 00002 | Loss 412025.1875 | Acc Train 0.5729 | Acc Test 0.5722
Epoch 00003 | Loss 389644.5938 | Acc Train 0.5753 | Acc Test 0.6201
Epoch 00004 | Loss 370667.1875 | Acc Train 0.6254 | Acc Test 0.5922
Epoch 00005 | Loss 354414.5000 | Acc Train 0.5960 | Acc Test 0.6332
Epoch 00006 | Loss 339332.8125 | Acc Train 0.6385 | Acc Test 0.6213
Epoch 00007 | Loss 326391.6875 | Acc Train 0.6266 | Acc Test 0.6305
Epoch 00008 | Loss 314656.7500 | Acc Train 0.6362 | Acc Test 0.6269
Epoch 00009 | Loss 304017.9688 | Acc Train 0.6323 | Acc Test 0.6288
Epoch 00010 | Loss 294472.9375 | Acc Train 0.6342 | Acc Test 0.6293
Epoch 00011 | Loss 285805.8125 | Acc Train 0.6345 | Acc Test 0.6372
Epoch 00012 | Loss 277988.0000 | Acc Train 0.6425 | Acc Test 0.6451
Epoch 00013 | Loss 270771.6250 | Acc Train 0.6515 | Acc Test 0.6503
Epoch 00014 | Loss 264275.0938 | Acc Train 0.655

AttributeError: ignored

In [ ]:
# g = model[0].batch_dgl_graph
# graph_list = dgl.unbatch(g)
# hidden_states = []
# max_nodes_num = max([graph.num_nodes() for graph in graph_list])
# for graph in graph_list:
#     hiddens = graph.ndata['h']
#     node_num, hidden_num = hiddens.size()
#     if len(hiddens) < max_nodes_num:
#         padding = hiddens.new_zeros(size=(max_nodes_num - node_num, hidden_num))
#         hiddens = torch.cat((hiddens, padding), dim=0)
#     hidden_states.append(hiddens)
# w = torch.stack(hidden_states)
# model[0].get_hidden_state()
# import numpy as np
l.shape

torch.Size([2, 2143, 7])

In [ ]:
for i in data['urls'][0:2]:
  c = data['labels'][i]
  print(len(c))


1910
2143


# Other

In [13]:
data = loadData()

tokenLength = 5000

In [4]:
from transformers import MarkupLMFeatureExtractor
feature_extractor = MarkupLMFeatureExtractor()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
lmdataset = []
for i in data['urls']:
  code = data['code'][i]
  f = feature_extractor(code)
  nodes = f['nodes']
  xpaths = f['xpaths']

  t = convertCodeToTree(code)
  res = {}

  def convertXPaths(t):
    res[t.xpath] = data['labels'][i][t.id]
    for child in t.children:
      convertXPaths(child)

  convertXPaths(t)


  node_labels = []
  for i in xpaths[0]:
    if (i in res):
      node_labels.append(res[i])
    else:
      node_labels.append(6)


  lmdataset.append({'nodes': nodes,
               'xpaths': xpaths,
               'node_labels': [node_labels], })



In [7]:
from torch.utils.data import Dataset

class MarkupLMDataset(Dataset):
    """Dataset for token classification with MarkupLM."""

    def __init__(self, data, processor=None, max_length=512):
        self.data = data
        self.processor = processor
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # first, get nodes, xpaths and node labels
        item = self.data[idx]
        nodes, xpaths, node_labels = item['nodes'], item['xpaths'], item['node_labels']

        # provide to processor
        encoding = self.processor(nodes=nodes, xpaths=xpaths, node_labels=node_labels,
                                  max_length=self.max_length, return_tensors="pt")

        # remove batch dimension
        encoding = {k: v.squeeze() for k, v in encoding.items()}

        return encoding


In [9]:
from transformers import MarkupLMProcessor, AutoProcessor, AutoModelForTokenClassification

processor = AutoProcessor.from_pretrained("microsoft/markuplm-base", max_length=tokenLength)
processor.parse_html = False
# dataset = MarkupLMDataset(data=lmdataset, processor=processor, max_length=tokenLength)
print(processor)


preprocessor_config.json:   0%|          | 0.00/100 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/5.53k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/55.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

MarkupLMProcessor:
- feature_extractor: MarkupLMFeatureExtractor {
  "feature_extractor_type": "MarkupLMFeatureExtractor",
  "processor_class": "MarkupLMProcessor"
}

- tokenizer: MarkupLMTokenizerFast(name_or_path='microsoft/markuplm-base', vocab_size=50265, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<

In [10]:
from torch.utils.data import DataLoader

dataloader = DataLoader(lmdataset, batch_size=1, shuffle=True)

In [7]:
# from transformers import AutoModelForTokenClassification, MarkupLMConfig

id2label = {0: "Navigation", 1:"Product Info", 2:"Product Versions", 3:"Product Images", 4: "Checkout Options", 5: "Shopping Cart", 6: "Other"}
label2id = {label:id for id, label in id2label.items()}

# configuration = MarkupLMConfig(max_position_embeddings=tokenLength, vocab_size=50266,id2label=id2label, label2id=label2id)

# model = AutoModelForTokenClassification.from_pretrained("microsoft/markuplm-base", config=configuration, ignore_mismatched_sizes=True)
# print(model)

In [8]:
# !pip3 install evaluate
# !pip3 install seqeval

import evaluate

label_list = ["B-" + x for x in list(id2label.values())]
# Metric
metric = evaluate.load("seqeval")

def get_labels(predictions, references):
    # Transform predictions and references tensos to numpy arrays
    if device.type == "cpu":
        y_pred = predictions.detach().clone().numpy()
        y_true = references.detach().clone().numpy()
    else:
        y_pred = predictions.detach().cpu().clone().numpy()
        y_true = references.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(pred, gold_label) if l != -100]
        for pred, gold_label in zip(y_pred, y_true)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(pred, gold_label) if l != -100]
        for pred, gold_label in zip(y_pred, y_true)
    ]
    return true_predictions, true_labels

def compute_metrics(return_entity_level_metrics=True):
    results = metric.compute()
    if return_entity_level_metrics:
        # Unpack nested dictionaries
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
import torch
from torch.optim import AdamW
from tqdm.auto import tqdm


optimizer = AdamW(model.parameters(), lr=5e-5)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

model.train()
for epoch in range(5):  # loop over the dataset multiple times
  # for i in lmdataset:
    # print(i)
    # inputs = processor(nodes=i["nodes"], xpaths=i["xpaths"], node_labels=i["node_labels"], return_tensors="pt", max_length=tokenLength)
  for batch in tqdm(dataloader):
    # get the inputs;
    print(batch)

    inputs = {k:v.to(device) for k,v in batch.items()}
        # for k, v in inputs.items():
            # print(v.shape)
        # print(inputs)

        # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = model(**inputs)

    loss = outputs.loss
    loss.backward()
    optimizer.step()

    print("Loss:", loss.item())

        # predictions = outputs.logits.argmax(dim=-1)
        # labels = batch["labels"]
        # preds, refs = get_labels(predictions, labels)
        # metric.add_batch(
        #     predictions=preds,
        #     references=refs,
        # )

        # eval_metric = compute_metrics()
        # print(f"Epoch {epoch}:", eval_metric)

  0%|          | 0/10 [00:00<?, ?it/s]

{'nodes': [[("Polo Ralph Lauren Core Classic Sport Cap - Macy's",), ('Skip to main content',), ('Get extra 30% off select styles! Code FRIEND.',), ('Shop Now',), ('Gift Registry',), ('Shop Your Store',), ('Sign In',), ('0',), ('Women',), ('Men',), ('Kids',), ('Home',), ('Beauty',), ('Shoes',), ('Handbags',), ('Jewelry',), ('Furniture & Mattresses',), ('Toys',), ('Gifts',), ('Trending',), ('Sale',), ("Women's Clothing",), ("All Women's Clothing",), ('New Arrivals',), ('Activewear',), ('Blazers',), ('Bras, Underwear & Lingerie',), ('100% Cashmere',), ('Coats & Jackets',), ('Dresses',), ('Hoodies & Sweatshirts',), ('Jeans',), ('Jumpsuits & Rompers',), ('Pajamas & Robes',), ('Pants & Capris',), ('Skirts',), ('Suits & Suit Separates',), ('Sweaters',), ('Swimsuits & Cover-Ups',), ('Tops',), ('Juniors',), ("All Juniors' Clothing",), ('Coats & Jackets',), ('Dresses',), ('Jeans',), ('Sweaters',), ('Tops',), ('Plus Sizes',), ('All Plus Size Clothing',), ('Coats & Jackets',), ('Dresses',), ('Pant

AttributeError: ignored

In [9]:
from transformers import AutoProcessor, AutoModelForTokenClassification, MarkupLMFeatureExtractor, MarkupLMConfig
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from tqdm.auto import tqdm


processor = AutoProcessor.from_pretrained("microsoft/markuplm-base", model_max_length=8000)
processor.parse_html = False
configuration = MarkupLMConfig(max_position_embeddings=8000, vocab_size=50266, num_labels=7)
model = AutoModelForTokenClassification.from_pretrained("microsoft/markuplm-base", config=configuration,ignore_mismatched_sizes=True)
model.train()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


feature_extractor = MarkupLMFeatureExtractor()
# encoding = feature_extractor(html)


# encoding = processor(nodes=encoding["nodes"], xpaths=encoding["xpaths"], node_labels=[node_labels], return_tensors="pt", max_length=8000)


data = []
for i in lmdataset[0:100]:
  data.append(processor(nodes=i["nodes"], xpaths=i["xpaths"], node_labels=i["node_labels"], return_tensors="pt", max_length=tokenLength))

from torch.utils.data import Dataset

class MarkupLMDataset(Dataset):
  """Dataset for token classification with MarkupLM."""

  def __init__(self, data, processor=None, max_length=512):
      self.data = data
      self.processor = processor
      self.max_length = max_length

  def __len__(self):
      return len(self.data)

  def __getitem__(self, idx):
      # first, get nodes, xpaths and node labels
      item = self.data[idx]
      # nodes, xpaths, node_labels = item['nodes'], item['xpaths'], item['node_labels']

      # provide to processor
      # encoding = self.processor(nodes=nodes, xpaths=xpaths, node_labels=node_labels,
                                # max_length=self.max_length, return_tensors="pt")

      # remove batch dimension
      encoding = item
      encoding = {k: v.squeeze() for k, v in encoding.items()}

      return encoding


dataset = MarkupLMDataset(data=data)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

epoch = 1

for batch in tqdm(dataloader):
  inputs = {k:v.to(device) for k,v in batch.items()}

  optimizer = AdamW(model.parameters(), lr=5e-5)

  optimizer.zero_grad()

  # with torch.no_grad():
  outputs = model(**inputs)

  loss = outputs.loss
  loss.backward()
  optimizer.step()

  print("Loss:", loss.item())


  predictions = outputs.logits.argmax(dim=-1)
  labels = batch["labels"]
  preds, refs = get_labels(predictions, labels)
  metric.add_batch(
      predictions=preds,
      references=refs,
  )

  eval_metric = compute_metrics()
  print(f"Epoch {epoch}:", eval_metric)

Some weights of MarkupLMForTokenClassification were not initialized from the model checkpoint at microsoft/markuplm-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of MarkupLMForTokenClassification were not initialized from the model checkpoint at microsoft/markuplm-base and are newly initialized because the shapes did not match:
- markuplm.embeddings.word_embeddings.weight: found shape torch.Size([50267, 768]) in the checkpoint and torch.Size([50266, 768]) in the model instantiated
- markuplm.embeddings.position_embeddings.weight: found shape torch.Size([514, 768]) in the checkpoint and torch.Size([8000, 768]) in the model instantiated
- markuplm.embeddings.token_type_embeddings.weight: found shape torch.Size([1, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-strea

Loss: 2.178149938583374
Epoch 1: {'Navigation_precision': 0.7894736842105263, 'Navigation_recall': 0.052878965922444184, 'Navigation_f1': 0.09911894273127753, 'Navigation_number': 851, 'Other_precision': 0.17777777777777778, 'Other_recall': 0.4161849710982659, 'Other_f1': 0.2491349480968858, 'Other_number': 173, 'Product Images_precision': 0.0, 'Product Images_recall': 0.0, 'Product Images_f1': 0.0, 'Product Images_number': 0, 'Product Info_precision': 0.0, 'Product Info_recall': 0.0, 'Product Info_f1': 0.0, 'Product Info_number': 2, 'Product Versions_precision': 0.0, 'Product Versions_recall': 0.0, 'Product Versions_f1': 0.0, 'Product Versions_number': 0, 'Shopping Cart_precision': 0.0, 'Shopping Cart_recall': 0.0, 'Shopping Cart_f1': 0.0, 'Shopping Cart_number': 1, 'overall_precision': 0.11392405063291139, 'overall_recall': 0.11392405063291139, 'overall_f1': 0.11392405063291139, 'overall_accuracy': 0.11392405063291139}


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
  2%|▏         | 2/100 [02:26<2:00:41, 73.90s/it]

Loss: 1.3324989080429077
Epoch 1: {'Navigation_precision': 0.7701357466063349, 'Navigation_recall': 1.0, 'Navigation_f1': 0.8701431492842536, 'Navigation_number': 851, 'Other_precision': 0.0, 'Other_recall': 0.0, 'Other_f1': 0.0, 'Other_number': 251, 'Product Info_precision': 0.0, 'Product Info_recall': 0.0, 'Product Info_f1': 0.0, 'Product Info_number': 2, 'Shopping Cart_precision': 0.0, 'Shopping Cart_recall': 0.0, 'Shopping Cart_f1': 0.0, 'Shopping Cart_number': 1, 'overall_precision': 0.7701357466063349, 'overall_recall': 0.7701357466063349, 'overall_f1': 0.7701357466063349, 'overall_accuracy': 0.7701357466063349}


  3%|▎         | 3/100 [03:58<2:12:44, 82.11s/it]

...

Loss: 0.194035142660141
Epoch 1: {'Checkout Options_precision': 0.0, 'Checkout Options_recall': 0.0, 'Checkout Options_f1': 0.0, 'Checkout Options_number': 2, 'Navigation_precision': 0.9976359338061466, 'Navigation_recall': 0.9917743830787309, 'Navigation_f1': 0.9946965232763701, 'Navigation_number': 851, 'Other_precision': 0.7653631284916201, 'Other_recall': 0.9927536231884058, 'Other_f1': 0.864353312302839, 'Other_number': 138, 'Product Info_precision': 0.0, 'Product Info_recall': 0.0, 'Product Info_f1': 0.0, 'Product Info_number': 4, 'Product Versions_precision': 0.0, 'Product Versions_recall': 0.0, 'Product Versions_f1': 0.0, 'Product Versions_number': 29, 'Shopping Cart_precision': 0.0, 'Shopping Cart_recall': 0.0, 'Shopping Cart_f1': 0.0, 'Shopping Cart_number': 1, 'overall_precision': 0.9570731707317073, 'overall_recall': 0.9570731707317073, 'overall_f1': 0.9570731707317073, 'overall_accuracy': 0.9570731707317073}


 99%|█████████▉| 99/100 [2:03:38<01:12, 72.47s/it]

Loss: 0.273801326751709
Epoch 1: {'Navigation_precision': 0.9239956568946797, 'Navigation_recall': 1.0, 'Navigation_f1': 0.9604966139954854, 'Navigation_number': 851, 'Other_precision': 0.9835164835164835, 'Other_recall': 0.7188755020080321, 'Other_f1': 0.8306264501160092, 'Other_number': 249, 'Product Info_precision': 0.0, 'Product Info_recall': 0.0, 'Product Info_f1': 0.0, 'Product Info_number': 2, 'Shopping Cart_precision': 0.0, 'Shopping Cart_recall': 0.0, 'Shopping Cart_f1': 0.0, 'Shopping Cart_number': 1, 'overall_precision': 0.9338168631006346, 'overall_recall': 0.9338168631006346, 'overall_f1': 0.9338168631006346, 'overall_accuracy': 0.9338168631006346}


100%|██████████| 100/100 [2:04:39<00:00, 74.80s/it]

Loss: 0.28582561016082764
Epoch 1: {'Checkout Options_precision': 0.0, 'Checkout Options_recall': 0.0, 'Checkout Options_f1': 0.0, 'Checkout Options_number': 2, 'Navigation_precision': 1.0, 'Navigation_recall': 0.9435957696827262, 'Navigation_f1': 0.9709794437726723, 'Navigation_number': 851, 'Other_precision': 0.6380090497737556, 'Other_recall': 1.0, 'Other_f1': 0.7790055248618785, 'Other_number': 141, 'Product Info_precision': 0.0, 'Product Info_recall': 0.0, 'Product Info_f1': 0.0, 'Product Info_number': 4, 'Product Versions_precision': 0.0, 'Product Versions_recall': 0.0, 'Product Versions_f1': 0.0, 'Product Versions_number': 25, 'Shopping Cart_precision': 0.0, 'Shopping Cart_recall': 0.0, 'Shopping Cart_f1': 0.0, 'Shopping Cart_number': 1, 'overall_precision': 0.921875, 'overall_recall': 0.921875, 'overall_f1': 0.921875, 'overall_accuracy': 0.921875}


In [12]:
import pickle

with open ('model.pkl', 'wb') as f:
  pickle.dump(model, f)

In [14]:
from transformers import AutoProcessor
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from tqdm.auto import tqdm


all_metrics = []
processor = AutoProcessor.from_pretrained("microsoft/markuplm-base", model_max_length=8000)
processor.parse_html = False


data = []
for i in lmdataset[100:]:
  data.append(processor(nodes=i["nodes"], xpaths=i["xpaths"], node_labels=i["node_labels"], return_tensors="pt", max_length=tokenLength))

from torch.utils.data import Dataset

class MarkupLMDataset(Dataset):
  """Dataset for token classification with MarkupLM."""

  def __init__(self, data, processor=None, max_length=512):
      self.data = data
      self.processor = processor
      self.max_length = max_length

  def __len__(self):
      return len(self.data)

  def __getitem__(self, idx):
      # first, get nodes, xpaths and node labels
      item = self.data[idx]

      encoding = item
      encoding = {k: v.squeeze() for k, v in encoding.items()}

      return encoding


dataset = MarkupLMDataset(data=data)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

epoch = 1

for batch in tqdm(dataloader):
  inputs = {k:v.to(device) for k,v in batch.items()}

  

  with torch.no_grad():
    outputs = model(**inputs)

  loss = outputs.loss

  print("Loss:", loss.item())
  predictions = outputs.logits.argmax(dim=-1)
  labels = batch["labels"]
  preds, refs = get_labels(predictions, labels)
  metric.add_batch(
      predictions=preds,
      references=refs,
  )

  eval_metric = compute_metrics()
  all_metrics.append(eval_metric)
  print(f"Epoch {epoch}:", eval_metric)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
  0%|          | 0/20 [00:00<?, ?it/s]